In [333]:
# import files
from threading import Thread
import queue as que
import pandas as pd
import os, pickle
import nltk
import math
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk
stemmer = SnowballStemmer("english")
cachedStopWords = stopwords.words("english")

In [415]:
# methods
def read(fileName):
    with open(fileName) as f:
        content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content]
    return content

def merge_two_dicts(x, y):
    z = x.copy()   # start with x's keys and values
    z.update(y)    # modifies z with y's keys and values & returns None
    return z

def seperator(docs):
    txt = []
    for line in range(0, len(docs)):
        words = docs[line].split()
        item = (",").join([words[0], (" ").join(words[1:])])
        txt.append(item)
    return txt

def write(data):
    with open('./training_data.csv', 'w') as f:
        for item in data:
            f.write("%s\n" % item)
            
            
def save_it_all(obj, filename):
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
    
def load_objects(file):
    with open(file, 'rb') as input:
        return pickle.load(input)
    
def vocabDict(vocab, dataframe):
    
    nk = 0
    n = dataframe["Word Positions"][dataframe["Type"]==docType].tolist()[0]
    probability = math.log(nk + 1) - math.log(n + len(vocab))

    newDict = {"count": 0, "probability":probability}
    vdict = dict((el,newDict) for el in vocab)
    return vdict

def wordCount(dataframe, vocab):
    for docType in dataframe['Type']:
        counts = [dict(Counter(dataframe["document"][dataframe["Type"]==docType].tolist()[0].split()))]
        emptyVocab = vocabDict(vocab, dataframe)
        dataframe["wordCount"][dataframe["Type"]==docType] = [merge_two_dicts(emptyVocab, counts[0])]
    
    return dataframe

def wordProbs(dataFrame):
    for docType in dataFrame["Type"]:
        wordCountDict = dataFrame["wordCount"][dataFrame["Type"]==docType].tolist()[0]
        for key in wordCountDict:
            nk = dataFrame["wordCount"][dataFrame["Type"]==docType].tolist()[0][key]
            n = dataFrame["Word Positions"][dataFrame["Type"]==docType].tolist()[0]
            probability = math.log(nk + 1) - math.log(n + len(vocabStopStemmingEdit))
            newDict = {"count": nk, "probability":probability}
            dataFrame["wordCount"][dataFrame["Type"]==docType].tolist()[0][key] = newDict
    return dataFrame
        

In [371]:
# main
# test extracting the vocabulary and data frame creation
# read file line by line 
fileName = "forumTraining.data"

# main:
data = read(fileName)
print(data[0])
sep_data = seperator(data)
print()
print(sep_data[0])
write(sep_data)


# columns should be the type and index should be the vocabulary


atheism alt atheism faq atheist resources archive name atheism resources alt atheism archive name resources last modified december version atheist resources addresses of atheist organizations usa freedom from religion foundation darwin fish bumper stickers and assorted other atheist paraphernalia are available from the freedom from religion foundation in the us write to ffrf p o box madison wi telephone evolution designs evolution designs sell the darwin fish it s a fish symbol like the ones christians stick on their cars but with feet and the word darwin written inside the deluxe moulded d plastic fish is postpaid in the us write to evolution designs laurel canyon north hollywood ca people in the san francisco bay area can get darwin fish from lynn gold try mailing figmo netcom com for net people who go to lynn directly the price is per fish american atheist press aap publish various atheist books critiques of the bible lists of biblical contradictions and so on one such book is the b


atheism,alt atheism faq atheist resources archive name atheism resources alt atheism archive name resources last modified december version atheist resources addresses of atheist organizations usa freedom from religion foundation darwin fish bumper stickers and assorted other atheist paraphernalia are available from the freedom from religion foundation in the us write to ffrf p o box madison wi telephone evolution designs evolution designs sell the darwin fish it s a fish symbol like the ones christians stick on their cars but with feet and the word darwin written inside the deluxe moulded d plastic fish is postpaid in the us write to evolution designs laurel canyon north hollywood ca people in the san francisco bay area can get darwin fish from lynn gold try mailing figmo netcom com for net people who go to lynn directly the price is per fish american atheist press aap publish various atheist books critiques of the bible lists of biblical contradictions and so on one such book is the 

In [299]:
# data frame set up 
df = pd.read_csv("./training_data.csv", sep=",", names=("Type","Document"))
df.head()
dataOriginal = df
# Docsj ← training documents for which the classification is cj
dataOriginal.head()

,Type,Document
0,atheism,alt atheism faq atheist resources archive name...
1,atheism,alt atheism faq introduction to atheism archiv...
2,atheism,re gospel dating in article mimsy umd edu mang...
3,atheism,re university violating separation of church s...
4,atheism,re soc motss et al princeton axes matching fun...


Collect all words occurring in the Sample documents
• Vocabulary ← set of all distinct words

In [301]:
# vocabulary = []
# for line in dataOriginal["Document"]:
#     for word in line.split():
#         if word not in vocabulary:
#             vocabulary.append(word)
# print(len(vocabulary))



73712


In [377]:
# save the vocabulary object
save_it_all(vocabulary, "./vocabulary.pkl")
vocabulary = load_objects("./vocabulary.pkl")
len(vocabulary)
"re" in vocabulary

True

For each class cj (document type) in C

• Docsj ← training documents for which the classification is cj

• Probability estimate of a particular class: P(cj) = |Docsj| / |training documents|

• Textj ← create a single document per class (concatenate all Docsj)

• n = total number of word positions in Textj

• For each word wk in Vocabulary nk = number of times wk occurs in Textj

• Estimate of word occurrence for particular document type:
    P(wk | cj) = (nk + 1) / (n + |Vocabulary|)

Probability of kth word in vocabulary, given a document of type j

In [304]:
# get the document types
types = list(set(dataOriginal["Type"].tolist()))
print(types)


['atheism', 'baseball', 'christianity', 'pc', 'guns', 'mideastpolitics', 'mac', 'electronics', 'forsale', 'autos', 'graphics', 'religion', 'xwindows', 'hockey', 'motorcycles', 'mswindows', 'cryptology', 'space', 'politics', 'medicine']


In [305]:
# get the total count of document types
counts = []
for docType in types:
    counts.append(len(df.loc[dataOriginal['Type'] == docType]))
print("Total document records: "+str(len(dataOriginal['Type'])))

dataframeCounts = pd.DataFrame({'Type':types, 'Count':counts})

dataframeCounts.head(20)

Total document records: 11293


,Type,Count
0,atheism,480
1,baseball,597
2,christianity,598
3,pc,590
4,guns,545
5,mideastpolitics,564
6,mac,578
7,electronics,591
8,forsale,585
9,autos,594


In [306]:
# set the probablility for that document occuring in the set, 
# Probability estimate of a particular class: P(cj) = |Docsj| / |training documents|
dataInfo = dataframeCounts
dataInfo['Probablility'] = None 
# dataInfo = dataInfo.drop(columns="Probablility")
for docType in dataInfo["Type"]:
    dataInfo["Probablility"][dataInfo['Type'] == docType] = dataInfo["Count"][dataInfo["Type"]==docType]/len(dataOriginal['Type'])

dataInfo.head()
    
    

,Type,Count,Probablility
0,atheism,480,0.0425042
1,baseball,597,0.0528646
2,christianity,598,0.0529532
3,pc,590,0.0522448
4,guns,545,0.04826


In [307]:
# Textj ← create a single document per class (concatenate all Docsj)
singleDocumentFrame = pd.DataFrame({'Type':types})
singleDocumentFrame["docData"] = None
for docType in dataInfo['Type']:
    singleDocumentFrame["docData"][singleDocumentFrame["Type"] == docType] = ' '.join(dataOriginal["Document"][dataOriginal["Type"] == docType].tolist())
    
    

dataOriginal # the original data set up no changes
vocabulary # the original vocab set of all the words in the original documents
dataInfo.head(5) # the counts and probability of the classes
singleDocumentFrame.head(5) # has all the athiesm docs in one spot. 


,Type,docData
0,atheism,alt atheism faq atheist resources archive name...
1,baseball,spring records the orioles pitching staff agai...
2,christianity,re sex education in article mar athos rutgers ...
3,pc,wanted bus card for logitech mouse i ve acquir...
4,guns,re gun control was re we re mad as hell at the...


In [311]:
# adding documents to other frame
dataInfo["document"] = None
for docType in dataInfo["Type"]:
    dataInfo["document"][dataInfo["Type"] == docType] = singleDocumentFrame["docData"][singleDocumentFrame["Type"]==docType]
    
dataInfo.head()
  
    

# n = total number of word positions in Textj
dataInfo["Word Positions"] = None
for docType in dataInfo["Type"]:
    dataInfo["Word Positions"][dataInfo["Type"] == docType] = len(dataInfo["document"][dataInfo["Type"]==docType].tolist()[0].split())
    
dataInfo.head(20)

dataInfo.to_csv("dataInfoOriginal.csv", index=False, sep=",", header=True)
dataInfo.head(20)

,Type,Count,Probablility,wordCount,document,Word Positions
0,atheism,480,0.0425042,None,alt atheism faq atheist resources archive name...,161442
1,baseball,597,0.0528646,None,spring records the orioles pitching staff agai...,119407
2,christianity,598,0.0529532,None,re sex education in article mar athos rutgers ...,216581
3,pc,590,0.0522448,None,wanted bus card for logitech mouse i ve acquir...,112583
4,guns,545,0.04826,None,re gun control was re we re mad as hell at the...,191723
5,mideastpolitics,564,0.0499424,None,re islam borders was israel misisipi to ganges...,272488
6,mac,578,0.0511821,None,re nutek faces apple s wrath article read in a...,97182
7,electronics,591,0.0523333,None,re help with tracking device in article fba e ...,115503
8,forsale,585,0.051802,None,motorcycle wanted followup to kedz wpi wpi edu...,69761
9,autos,594,0.052599,None,re saturn s pricing policy in article c vir l ...,127696


In [312]:
# For each word wk in Vocabulary nk = number of times wk occurs in Textj
for docType in dataInfo['Type']:
    dataInfo["wordCount"][dataInfo["Type"]==docType] = [dict(Counter(dataInfo["document"][dataInfo["Type"]==docType].tolist()[0].split()))]


In [ ]:
# save the thing
dataInfo.to_csv("dataInfoOriginal.csv", index=False, sep=",", header=True)
save_it_all(dataInfo, "./dataInfo.pkl")

In [344]:
# load objects
dataInfo = load_objects("./dataInfo.pkl")
dataInfo.head()

,Type,Count,Probablility,wordCount,document,Word Positions
0,atheism,480,0.0425042,"{'alt': 89, 'atheism': 301, 'faq': 66, 'atheis...",alt atheism faq atheist resources archive name...,161442
1,baseball,597,0.0528646,"{'spring': 40, 'records': 6, 'the': 5561, 'ori...",spring records the orioles pitching staff agai...,119407
2,christianity,598,0.0529532,"{'re': 598, 'sex': 47, 'education': 30, 'in': ...",re sex education in article mar athos rutgers ...,216581
3,pc,590,0.0522448,"{'wanted': 30, 'bus': 316, 'card': 378, 'for':...",wanted bus card for logitech mouse i ve acquir...,112583
4,guns,545,0.04826,"{'re': 705, 'gun': 975, 'control': 290, 'was':...",re gun control was re we re mad as hell at the...,191723


In [318]:
# access word count in dataframe dict
dataInfo["wordCount"][dataInfo["Type"]==docType].tolist()[0]['info']

,Type,Count,Probablility,wordCount,document,Word Positions
0,atheism,480,0.0425042,"{'alt': 89, 'atheism': 301, 'faq': 66, 'atheis...",alt atheism faq atheist resources archive name...,161442
1,baseball,597,0.0528646,"{'spring': 40, 'records': 6, 'the': 5561, 'ori...",spring records the orioles pitching staff agai...,119407
2,christianity,598,0.0529532,"{'re': 598, 'sex': 47, 'education': 30, 'in': ...",re sex education in article mar athos rutgers ...,216581
3,pc,590,0.0522448,"{'wanted': 30, 'bus': 316, 'card': 378, 'for':...",wanted bus card for logitech mouse i ve acquir...,112583
4,guns,545,0.04826,"{'re': 705, 'gun': 975, 'control': 290, 'was':...",re gun control was re we re mad as hell at the...,191723


In [358]:
# remove stop words
# dataInfoStopEdit = dataInfo.copy()
# for docType in dataInfoStopEdit["Type"]:
#     print(docType)
#     text = dataInfoStopEdit["document"][dataInfoStopEdit["Type"]==docType].tolist()[0]
#     dataInfoStopEdit["document"][dataInfoStopEdit["Type"]==docType] = ' '.join([word for word in text.split() if word not in cachedStopWords])

#     dataInfoStopEdit["Word Positions"][dataInfoStopEdit["Type"]==docType] = len(dataInfoStopEdit["document"][dataInfoStopEdit["Type"]==docType].tolist()[0].split())

# dataInfoStopEdit = wordCount(dataInfoStopEdit)

# # # save the thing
# # dataInfoStopEdit.to_csv("dataInfoStopEdit.csv", index=False, sep=",", header=True)
# # save_it_all(dataInfoStopEdit, "./dataInfoStopEdit.pkl")

dataInfoStopEdit.head()


vocabStopEdit = []
vocabStopEdit = set([word for word in vocabulary if word not in cachedStopWords])
len(vocabStopEdit)
#save_it_all(vocabStopEdit, "./vocabStemmingEdit.pkl")
len(vocabStopEdit)

73560

In [356]:
# remove Stemming words

dataInfoStemmingEdit = dataInfo.copy()
for docType in dataInfoStemmingEdit["Type"]:
    print(docType)
    text = dataInfoStemmingEdit["document"][dataInfoStemmingEdit["Type"]==docType].tolist()[0]
    dataInfoStemmingEdit["document"][dataInfoStemmingEdit["Type"]==docType] = ' '.join([stemmer.stem(word) for word in text.split() ])

    dataInfoStemmingEdit["Word Positions"][dataInfoStemmingEdit["Type"]==docType] = len(dataInfoStemmingEdit["document"][dataInfoStemmingEdit["Type"]==docType].tolist()[0].split())

dataInfoStemmingEdit = wordCount(dataInfoStemmingEdit)

# # save the thing
# dataInfoStemmingEdit.to_csv("dataInfoStemmingEdit.csv", index=False, sep=",", header=True)
# save_it_all(dataInfoStemmingEdit, "./dataInfoStemmingEdit.pkl")
dataInfoStemmingEdit.head()

vocabStemmingEdit = []
vocabStemmingEdit = set([stemmer.stem(word) for word in vocabulary])
len(vocabStemmingEdit)
#save_it_all(vocabStemmingEdit, "./vocabStemmingEdit.pkl")


In [416]:
# remove stop and stemming words
dataInfoStopStemmingEdit = dataInfo.copy()

vocabStopStemmingEdit = []
vocabStopStemmingEdit = [stemmer.stem(word) for word in vocabulary if word not in cachedStopWords]

for docType in dataInfoStopStemmingEdit["Type"]:
    print(docType)
    text = dataInfoStopStemmingEdit["document"][dataInfoStopStemmingEdit["Type"]==docType].tolist()[0]
    dataInfoStopStemmingEdit["document"][dataInfoStopStemmingEdit["Type"]==docType] = ' '.join([stemmer.stem(word) for word in text.split() if word not in cachedStopWords])

    dataInfoStopStemmingEdit["Word Positions"][dataInfoStopStemmingEdit["Type"]==docType] = len(dataInfoStopStemmingEdit["document"][dataInfoStopStemmingEdit["Type"]==docType].tolist()[0].split())

dataInfoStopStemmingEdit = wordCount(dataInfoStopStemmingEdit, vocabStopStemmingEdit)

# save the thing
dataInfoStopStemmingEdit.to_csv("dataInfoStopStemmingEdit.csv", index=False, sep=",", header=True)
save_it_all(dataInfoStopStemmingEdit, "./dataInfoStopStemmingEdit.pkl")
dataInfoStopStemmingEdit.head()
len(set(vocabStopStemmingEdit))
save_it_all(vocabStopStemmingEdit, "./vocabStopStemmingEdit.pkl")
#
dataInfoStopStemmingEdit.head()



atheism
baseball
christianity
pc
guns
mideastpolitics
mac
electronics
forsale
autos
graphics
religion
xwindows
hockey
motorcycles
mswindows
cryptology
space
politics
medicine


,Type,Count,Probablility,wordCount,document,Word Positions
0,atheism,480,0.0425042,"{'alt': 89, 'atheism': 301, 'faq': 73, 'atheis...",alt atheism faq atheist resourc archiv name at...,81587
1,baseball,597,0.0528646,"{'alt': 1, 'atheism': {'count': 0, 'probabilit...",spring record oriol pitch staff fine exhibit s...,66918
2,christianity,598,0.0529532,"{'alt': 14, 'atheism': 17, 'faq': 17, 'atheist...",sex educ articl mar atho rutger edu swansond n...,105784
3,pc,590,0.0522448,"{'alt': 10, 'atheism': {'count': 0, 'probabili...",want bus card logitech mous acquir old logitec...,64961
4,guns,545,0.04826,"{'alt': 11, 'atheism': {'count': 0, 'probabili...",gun control mad hell tv news hambidg bms com w...,103158


In [418]:
# word probablity 
# work count probability for dataInfoStopStemmingEdit
dataInfoStopStemmingEdit = load_objects("dataInfoStopStemmingEdit.pkl")
for docType in dataInfoStopStemmingEdit["Type"]:
    print(docType)
    wordCountDict = list(set(dataInfoStopStemmingEdit["document"][dataInfoStopStemmingEdit["Type"]==docType].tolist()[0].split()))
    for key in wordCountDict:
        nk = dataInfoStopStemmingEdit["wordCount"][dataInfoStopStemmingEdit["Type"]==docType].tolist()[0][key]
        n = dataInfoStopStemmingEdit["Word Positions"][dataInfoStopStemmingEdit["Type"]==docType].tolist()[0]
        probability = math.log(nk + 1) - math.log(n + len(vocabStopStemmingEdit))
        newDict = {"count": nk, "probability":probability}
        dataInfoStopStemmingEdit["wordCount"][dataInfoStopStemmingEdit["Type"]==docType].tolist()[0][key] = newDict



atheism
baseball
christianity
pc
guns
mideastpolitics
mac
electronics
forsale
autos
graphics
religion
xwindows
hockey
motorcycles
mswindows
cryptology
space
politics
medicine


In [427]:
dataInfoStopStemmingEdit.head()
save_it_all(dataInfoStopStemmingEdit, "./dataInfoStopStemmingEdit.pkl")

In [426]:
dataInfoStopStemmingEdit["wordCount"][dataInfoStopStemmingEdit["Type"]==docType].tolist()


[{'alt': {'count': 22, 'probability': -8.908035970414664},
  'atheism': {'count': 0, 'probability': -12.043530186343814},
  'faq': {'count': 37, 'probability': -8.405944026617428},
  'atheist': {'count': 0, 'probability': -12.043530186343814},
  'resourc': {'count': 18, 'probability': -9.099091207177374},
  'archiv': {'count': 18, 'probability': -9.099091207177374},
  'name': {'count': 60, 'probability': -7.932656322170502},
  'last': {'count': 76, 'probability': -7.69972476449013},
  'modifi': {'count': 5, 'probability': -10.25177071711576},
  'decemb': {'count': 9, 'probability': -9.740945093349769},
  'version': {'count': 30, 'probability': -8.609542981858667},
  'address': {'count': 43, 'probability': -8.259340552425552},
  'organ': {'count': 89, 'probability': -7.543720516013549},
  'usa': {'count': 15, 'probability': -9.270941464104032},
  'freedom': {'count': 2, 'probability': -10.944917897675705},
  'religion': {'count': 0, 'probability': -12.043530186343814},
  'foundat': {'co

In [432]:
len(vocabStopStemmingEdit)
"worshipped" in vocabStopEdit

True